<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Анализ-зависимости" data-toc-modified-id="Анализ-зависимости-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ зависимости</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Update:" data-toc-modified-id="Update:-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Update:</a></span></li></ul></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

In [1]:
#conda install -c conda-forge seaborn 

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/Users/AlexeyKrutilin/Desktop/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
13084,1,-3063.670868,27,среднее,1,женат / замужем,0,M,сотрудник,0,201511.253332,операции с коммерческой недвижимостью
3321,0,343060.322658,63,высшее,0,женат / замужем,0,F,пенсионер,0,208918.420517,приобретение автомобиля
6629,1,-665.701337,26,ВЫСШЕЕ,0,женат / замужем,0,F,сотрудник,0,134420.239376,покупка своего жилья
14360,0,-4803.610046,58,высшее,0,в разводе,3,M,сотрудник,0,169553.747930,строительство собственной недвижимости
14973,0,-1229.633142,49,среднее,1,гражданский брак,1,F,сотрудник,0,153517.982011,приобретение автомобиля
9475,0,-1970.538080,32,среднее,1,гражданский брак,1,F,сотрудник,0,87512.779262,свадьба
12940,0,-5891.373444,41,Среднее,1,женат / замужем,0,M,сотрудник,0,101228.770516,покупка жилья для сдачи
14388,0,-4415.934458,36,среднее,1,Не женат / не замужем,4,F,компаньон,0,273863.635149,операции с жильем
10896,1,-1244.185012,32,среднее,1,женат / замужем,0,M,сотрудник,0,179265.330385,покупка жилой недвижимости
13991,0,-1366.827960,25,среднее,1,Не женат / не замужем,4,F,сотрудник,0,181401.259243,профильное образование


Видно, что данные имеют пропуски только в колонках **days_employed** и **total_income**, причем количество пропусков совпадает. Есть предположение, что каждая строка, где есть пропуск имеет 2 пропуска одновременно, причем по указанным выше столбцам. Скорее всего пропуски были сделаны, так как клиент не предоставил данные о своей занятости.

In [5]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Видим, что есть отрицательные значения в столбцах, где по смыслу таковых быть не должно(возможно, человек при вводе писал некоторые значения через тире).<br>
Такие столбцы, как: **children**, **days_employed**.

**Вывод**

Из столбца **children** можно заметить, что у одного из клиентов 20 детей, что довольно внушительно. Возможно, этот клиент окажется артефактом для выборки.  
Также, довольно подозрительным является клиент с трудовым стажем в более, чем 1000 лет. Причем, этот клиент скорее всего не единичный случай, так как среднее значение по столбцу **days_employed** вскачило аж до 175 лет, что явно больше средней продолжительности жизни, а тем более трудоспособного возраста.    
Видно из столбца с возрастом клиента **dob_years**, что присутствуют люди, подавшие на кредит в 0 лет.  
***Данные нуждаются в дальнейшем исследовании.***

## Предобработка данных

### Обработка пропусков

Для начала заменим отрицательные значения положительными:

In [6]:
df.loc[ df['children'] < 0, 'children'] = (-1) * df.loc[ df['children'] < 0, 'children']
df.loc[ df['days_employed'] < 0, 'days_employed'] = (-1) * df.loc[ df['days_employed'] < 0, 'days_employed']
df.describe()
#df['children'] = df['children'].abs()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.543275,66914.728907,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.379876,139030.880527,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Обработаем пропуски:

In [7]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Убедимся, что пропуски в строках появляются только парами:

In [8]:
df[ df['days_employed'].isna() & df['total_income'].isna() ].isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Отсюда следует вывод, что данные о работе еще не предоставлены, поэтому нет ни информации о стаже, ни информации о доходе для одних и тех же людей.

Так как пропуски стоят в количественных переменных, то замена на значение *'unknown'* до момента выяснения данных не подойдет.  
Столбец days_employed содержит выбросы -> среднее значение не показательно. Заменим пропуски медианными значениями.

In [9]:
df['income_type'].value_counts().to_frame()

,income_type
сотрудник,11119
компаньон,5085
пенсионер,3856
госслужащий,1459
безработный,2
предприниматель,2
студент,1
в декрете,1


Сгруппируем заемщиков по какому-либо признаку, например, по типу занятости. А затем в каждой группе найдем медианную зарплату. Ведь логично, что уровень дохода зависит от типа занятости. Удобнее всего произвести такую замену с помощью метода transform():

In [10]:
#df['days_employed'] = df['days_employed'].fillna( df['days_employed'].median() )
#df['total_income'] = df['total_income'].fillna( df['total_income'].median() )
#df.isna().sum()

df['days_employed'] = df['days_employed'].fillna( df.groupby('income_type')['days_employed'].transform('median'))
df['total_income'] = df['total_income'].fillna( df.groupby('income_type')['total_income'].transform('median'))
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Посмотрим, получится ли угадать неявные пропуски:

In [11]:
df.loc[(df['days_employed'] == 'None') | 
       (df['days_employed'] == 'NaN') |
       (df['days_employed'] == ' ') |
       (df['days_employed'] == '-')].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          0 non-null      int64  
 1   days_employed     0 non-null      float64
 2   dob_years         0 non-null      int64  
 3   education         0 non-null      object 
 4   education_id      0 non-null      int64  
 5   family_status     0 non-null      object 
 6   family_status_id  0 non-null      int64  
 7   gender            0 non-null      object 
 8   income_type       0 non-null      object 
 9   debt              0 non-null      int64  
 10  total_income      0 non-null      float64
 11  purpose           0 non-null      object 
dtypes: float64(2), int64(5), object(5)
memory usage: 0.0+ bytes


/Users/AlexeyKrutilin/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [12]:
df.replace(['None','NaN',' ','-'], None).isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Неявных пропусков не обнаружено.

**Вывод**

Пропуски успешно заполнены, неявные пропуски не обнаружены.

### Замена типа данных

In [13]:
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

В столбце **days_employed** нам нужны целочисленные переменные.  
Метод **to_numeric** не подойдет, потому что он переводит все числа в тип *float*.  
Воспользуемся методом **astype**.  

In [14]:
df['days_employed'] = df['days_employed'].astype('int')
df.dtypes

children              int64
days_employed         int64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [15]:
df['total_income'] = df['total_income'].astype('int')
df.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

**Вывод**

Теперь в датафрейме данные нужных типов.

### Обработка дубликатов

In [16]:
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3991,0,332966,53,Высшее,0,вдовец / вдова,2,F,пенсионер,0,156055,высшее образование
6532,0,648,36,среднее,1,в разводе,3,F,сотрудник,0,124197,на покупку подержанного автомобиля
16259,0,2326,47,среднее,1,гражданский брак,1,F,сотрудник,0,161380,дополнительное образование
1795,0,390745,66,среднее,1,гражданский брак,1,F,пенсионер,0,129347,свадьба
13957,0,8461,52,Среднее,1,женат / замужем,0,F,госслужащий,1,130000,дополнительное образование


In [17]:
df.duplicated().sum()

54

Проверим все признаки, в колонках которых есть наборы букв:

In [18]:
for i in [3,5,8,11]:
    df.iloc[:,i] = df.iloc[:,i].str.lower()
#циклы в pandas - плохо, как иначе нормально проверить - непонятно

df.duplicated().sum()

71

Имеем 71 дубликат.  
Тут можно было воспользоваться методом **value_counts()**, но так как нет id пользователей - его не к чему применять.

In [19]:
df[ df.duplicated(keep=False) ].sample(7)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2326,41,среднее,1,женат / замужем,0,F,сотрудник,0,161380,покупка жилья для семьи
6312,0,2326,30,среднее,1,женат / замужем,0,M,сотрудник,0,161380,строительство жилой недвижимости
19832,0,2326,48,среднее,1,женат / замужем,0,F,сотрудник,0,161380,ремонт жилью
19295,0,2326,51,среднее,1,женат / замужем,0,F,сотрудник,0,161380,покупка коммерческой недвижимости
19041,0,2326,56,среднее,1,гражданский брак,1,F,сотрудник,0,161380,на проведение свадьбы
19946,0,2326,57,среднее,1,женат / замужем,0,F,сотрудник,0,161380,сделка с подержанным автомобилем
18428,0,365003,64,среднее,1,женат / замужем,0,F,пенсионер,0,137127,жилье


У всех дубликатов один и тот же трудовой стаж и все они женского пола, общая прибыль тоже совпадает, отсюда следует, что это один и тот же человек в разные моменты времени!<br> 
Попробуем оставить строчку с максимальным возрастом (последний приход):

In [20]:
print(df[ df.duplicated(keep=False) ].sort_values('dob_years', ascending=False)['dob_years'].unique())

last_seen_age = df[ df.duplicated(keep=False) ]['dob_years'].max()
print(f'Последний входящий возраст: {last_seen_age} год')

[71 66 65 64 62 61 60 59 58 57 56 54 51 50 48 47 46 45 44 43 42 41 40 39
 38 37 36 35 34 32 31 30 29 23]
Последний входящий возраст: 71 год


Оставим только сторочку, где возраст равен 71:

In [21]:
stub_df = df[ (df.duplicated(keep=False)) & (df['dob_years'] == last_seen_age) ]
stub_df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6537,0,365003,71,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
7938,0,365003,71,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
9604,0,365003,71,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы


In [22]:
stubborn_person = stub_df.loc[9604,:]
stubborn_person.to_frame()

,9604
children,0
days_employed,365003
dob_years,71
education,среднее
education_id,1
family_status,гражданский брак
family_status_id,1
gender,F
income_type,пенсионер
debt,0


In [23]:
df.drop_duplicates(keep=False, inplace=True)
df.loc[9604] = stubborn_person
df.loc[9604].to_frame()

,9604
children,0
days_employed,365003
dob_years,71
education,среднее
education_id,1
family_status,гражданский брак
family_status_id,1
gender,F
income_type,пенсионер
debt,0


In [24]:
df.duplicated().sum()

0

**Вывод**

Все дубликаты принадлежали одному человеку, дубликаты почищены.

### Лемматизация

Проведем лемматизацию столбца с целью получения кредита:

In [25]:
from pymystem3 import Mystem
m = Mystem()
for i in df.index:
    df.loc[i, 'purpose'] = ' '.join(m.lemmatize(df.loc[i, 'purpose']))
df['purpose'].head(10)
#цикл работает долго, интересно, можно ли как-то оптимизировать процесс

0                  покупка   жилье \n
1        приобретение   автомобиль \n
2                  покупка   жилье \n
3     дополнительный   образование \n
4                сыграть   свадьба \n
5                  покупка   жилье \n
6             операция   с   жилье \n
7                      образование \n
8        на   проведение   свадьба \n
9    покупка   жилье   для   семья \n
Name: purpose, dtype: object


<div class="alert alert-info" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий <a class="tocSkip"></a></h2>
    
Альтернативно, лемматизацию можно было бы произвести так:<br>
    
    from pymystem3 import Mystem
    m = Mystem()
    def lemmatize_text(row):
        purpose = row['purpose']
        return m.lemmatize(purpose)
    df['purpose'] = df.apply(lemmatize_text, axis=1)
    
</div> 

In [26]:
lemmas = [] 
for i in df['purpose'].unique():
    lemma = ' '.join(m.lemmatize(i))
    lemmas.append(lemma)
lemmas

['покупка     жилье  \n',
 'приобретение     автомобиль  \n',
 'дополнительный     образование  \n',
 'сыграть     свадьба  \n',
 'операция     с     жилье  \n',
 'образование  \n',
 'на     проведение     свадьба  \n',
 'покупка     жилье     для     семья  \n',
 'покупка     недвижимость  \n',
 'покупка     коммерческий     недвижимость  \n',
 'покупка     жилой     недвижимость  \n',
 'строительство     собственный     недвижимость  \n',
 'недвижимость  \n',
 'строительство     недвижимость  \n',
 'на     покупка     подержать     автомобиль  \n',
 'на     покупка     свой     автомобиль  \n',
 'операция     с     коммерческий     недвижимость  \n',
 'строительство     жилой     недвижимость  \n',
 'жилье  \n',
 'операция     со     свой     недвижимость  \n',
 'автомобиль  \n',
 'заниматься     образование  \n',
 'сделка     с     подержанный     автомобиль  \n',
 'получение     образование  \n',
 'свадьба  \n',
 'получение     дополнительный     образование  \n',
 'покупка     сво

 #### Вывод

Провели лемматизацию столбца отдельно для каждой ячейки в цикле.

### Категоризация данных

In [27]:
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1326,1,363065,62,высшее,0,вдовец / вдова,2,F,пенсионер,0,152775,покупка жилье \n
1901,0,3678,37,среднее,1,женат / замужем,0,F,сотрудник,0,138535,на покупка свой автомобиль \n
6100,3,3262,42,среднее,1,женат / замужем,0,F,сотрудник,0,96606,получение дополнительный образование \n
16871,0,220,50,среднее,1,женат / замужем,0,M,сотрудник,0,86890,покупка жилье \n
18923,0,950,25,высшее,0,не женат / не замужем,4,F,компаньон,0,276681,строительство собственный недвижимость \n


Выделим словари и логи из датафрейма:

In [28]:
education_dict = df[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.head()

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [29]:
family_status_dict = df[['family_status_id','family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict.head()

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [30]:
df_logs = df.drop(['education','family_status'], axis='columns')
df_logs.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилье \n
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиль \n
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилье \n
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительный образование \n
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьба \n


**Вывод**

В новой таблице **df_logs** теперь нет столбцов с единичной корреляцией, при этом человек может получить полную информацию о таблице, обратившись к только что созданным словарям.

## Анализ зависимости

Проверим, есть ли зависимость между наличием детей и возвратом кредита в срок

In [31]:
children_debt = df.groupby('children').agg({'debt':'sum'})
children_debt.head(7)

,debt
children,
0,1063
1,445
2,194
3,27
4,4
5,0
20,8


In [32]:
def cat_children(row):
    if row['children'] == 0:
        return 'has no children'
    return 'has children'

df_logs['children_cat'] = df_logs.apply(cat_children, axis=1)
children_debt_cat = df_logs.groupby('children_cat').agg({'debt':'sum'})
children_debt_cat.sort_values(by='debt',ascending=False)

,debt
children_cat,
has no children,1063
has children,678


Соотнесем с тем, сколько людей относятся к определенной категории и добавим этот столбец:

In [33]:
children_debt_cat['total'] = df_logs['children_cat'].value_counts()
children_debt_cat

,debt,total
children_cat,,
has children,678,7350
has no children,1063,14039


In [34]:
children_debt_cor = children_debt_cat['debt'] / children_debt_cat['total']
children_debt_cor.to_frame()

,0
children_cat,
has children,0.092245
has no children,0.075718


**Вывод**

Тем у кого дети есть, судя по предоставленым данным, немного более свойственно не возвращать долг.  
Однако отличия несущественны -> явной зависимости нет.

Update:

In [35]:
df_logs.pivot_table(index = 'children', values = 'debt', aggfunc = 'mean')

,debt
children,
0,0.075718
1,0.091847
2,0.094680
3,0.081818
4,0.097561
5,0.000000
20,0.105263


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [36]:
status_debt = df.groupby('family_status_id').agg({'debt':'sum'})
status_debt

,debt
family_status_id,
0,931
1,388
2,63
3,85
4,274


На первый взгляд, людям в браке более свойственно иметь долги.

Посмотрим, как часто встречается каждая категория в датафрейме:

In [37]:
status_debt['total'] = df_logs.groupby('family_status_id')['family_status_id'].count()
status_debt['total']

family_status_id
0    12300
1     4129
2      958
3     1195
4     2807
Name: total, dtype: int64

Посмотрим, какой процент людей из каждой категории возвращает долг:

In [38]:
status_debt_cor = status_debt['debt'] / status_debt['total']
#status_debt_cor.rename(columns = 'corr', inplace = True)
#как заменить название столбца с 0 на что-то более внятное?
status_debt_cor.to_frame()

,0
family_status_id,
0,0.075691
1,0.093969
2,0.065762
3,0.071130
4,0.097613


**Вывод**

Зависимость есть.  
Вдовы/вдовцы менее склонны к задолжностям, чем другие категории.  
Наиболее склонные - люди, состоящие в браке или в него не вступавшие.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Разделим данные на 4 категории:

In [39]:
def cat_income(row):
    if row['total_income'] <= 60000:
        return 'небольшой'
    elif 60000 < row['total_income'] <= 120000:
        return 'средний'
    elif 120000 < row['total_income'] <= 300000:
        return 'высокий'
    return 'огромный'

df_logs['total_income_cat'] = df_logs.apply(cat_income, axis=1)
income_debt_cat = df_logs.groupby('total_income_cat').agg({'debt':'sum'})
income_debt_cat

,debt
total_income_cat,
высокий,1084
небольшой,49
огромный,106
средний,502


In [40]:
income_debt_cat['total'] = df_logs.groupby('total_income_cat')['total_income_cat'].count()
income_debt_cat

,debt,total
total_income_cat,,
высокий,1084,13061
небольшой,49,806
огромный,106,1483
средний,502,6039


In [41]:
income_debt_corr = income_debt_cat['debt'] / income_debt_cat['total']
income_debt_corr

total_income_cat
высокий      0.082995
небольшой    0.060794
огромный     0.071477
средний      0.083126
dtype: float64

#### Update:   
Разделим *total_income* по квантилям:

In [42]:
df_logs = df_logs.reset_index(drop=True)
df_logs['total_income'] = df_logs['total_income'].astype('int')
df_logs['total_income_cat2'] = pd.qcut(df_logs['total_income'], 4,
                                            labels=['небольшой', 'средний', 'высокий', 'огромный'])
df_logs['total_income_cat2'].to_frame()

,total_income_cat2
0,огромный
1,средний
2,средний
3,огромный
4,высокий
...,...
21384,высокий
21385,небольшой
21386,огромный
21387,небольшой


In [43]:
income_debt_cat['total2'] = df_logs.groupby('total_income_cat2')['total_income_cat2'].count()
income_debt_cat['debt2'] = df_logs.groupby('total_income_cat2')['debt'].sum()
income_debt_cat

,debt,total,total2,debt2
total_income_cat,,,,
высокий,1084,13061,5785,494
небольшой,49,806,5348,426
огромный,106,1483,4909,345
средний,502,6039,5347,476


In [44]:
income_debt_corr2 = income_debt_cat['debt2'] / income_debt_cat['total2']
income_debt_corr2

total_income_cat
высокий      0.085393
небольшой    0.079656
огромный     0.070279
средний      0.089022
dtype: float64

Отличия всё равно есть, даже когда разделили по квантилитам (хоть количество в них почему-то сильно разнится)

**Вывод**

Зависимость есть.  
Люди с доходом от 60.000 до 300.000 наиболее склонны к задолжностям.

- Как разные цели кредита влияют на его возврат в срок?

In [45]:
purpose_debt = df_logs.groupby('purpose').agg({'debt':'sum'})
purpose_debt

,debt
purpose,
автомобиль \n,86
высокий образование \n,40
дополнительный образование \n,38
жилье \n,46
заниматься высокий образование \n,43
заниматься образование \n,39
на покупка автомобиль \n,44
на покупка подержать автомобиль \n,36
на покупка свой автомобиль \n,46


Большое количество целей, разобьем на более глобальные группы:

In [46]:
def purpose_stem(row):
    if 'автомобиль' in row['purpose']:
        return 'автомобиль'
    elif 'образование' in row['purpose']:
        return 'образование'
    elif 'жилье' in row['purpose']:
        return 'жилье'
    elif 'свадьба' in row['purpose']:
        return 'свадьба'
    elif 'недвижимость' in row['purpose']:
        return 'недвижимость'

df_logs['purpose_stem'] = df_logs.apply(purpose_stem, axis=1)
purpose_debt_cat = df_logs.groupby('purpose_stem').agg({'debt':'sum'})
purpose_debt_cat

,debt
purpose_stem,
автомобиль,403
жилье,308
недвижимость,474
образование,370
свадьба,186


На первый взгляд должников меньше всего среди тех, кто берет кредит на свадьбу.  
Подсчитаем число для каждой категории:

In [47]:
purpose_debt_cat['total'] = df_logs.groupby('purpose_stem')['purpose_stem'].count()
purpose_debt_cat

,debt,total
purpose_stem,,
автомобиль,403,4297
жилье,308,4449
недвижимость,474,6335
образование,370,4004
свадьба,186,2304


In [48]:
purpose_debt_corr = purpose_debt_cat['debt'] / purpose_debt_cat['total']
purpose_debt_corr

purpose_stem
автомобиль      0.093786
жилье           0.069229
недвижимость    0.074822
образование     0.092408
свадьба         0.080729
dtype: float64

**Вывод**

Зависимость есть.  
Большая часть должников среди автолюбителей и людей, берущих кредит на образование, а меньше всего - берущих деньги на жильё.  
Однако гипотеза, что люди, берущие деньги на свадьбу - наиболее склонные к своевременному возврату, не подтвердилась.

##  Общий вывод

Те, у кого есть дети, чуть менее вероятно возвращают долг.  
Однако семейное положение играет более существенную роль: люди, никогда не пребывавшие в браке, более склонны иметь долги, чем те, кто потерял супруга.

## Чек-лист готовности проекта



- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.